In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import load_model

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = keras.preprocessing.image_dataset_from_directory('../input/melspec-new-train/MelSpectro_Classification_1Sec',
                                                 validation_split=0.1,
                                                 label_mode = "categorical",
                                                 subset="training",
                                                 seed=123,
                                                 image_size = (64, 64),
                                                 batch_size = 64)

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = keras.preprocessing.image_dataset_from_directory('../input/melspec-new-train/MelSpectro_Classification_1Sec',
                                                 validation_split=0.1,
                                                 label_mode = "categorical",
                                                 subset="validation",
                                                 seed=123,
                                                 image_size = (64, 64),
                                                 batch_size = 64)

In [ ]:
# Sequential 2-D CNN Model
cnn = tf.keras.models.Sequential() 
# Adding & Pooling Conv.Layer 1
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# Adding & Pooling Conv. Layer 2 
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# Adding & Pooling Conv. Layer 3
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# Flattening 
cnn.add(tf.keras.layers.Flatten())
# Adding 3 hidden layers
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
# O/P Layer
cnn.add(tf.keras.layers.Dense(units=len(training_set.class_names), activation='softmax'))
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
h1 = cnn.fit(x = training_set, validation_data = test_set, epochs = 5)

In [ ]:
cnn.save("MelSpec3_Run1_Eve")

In [ ]:
new_model = tf.keras.models.load_model("MelSpec3_Run1_Eve")




In [ ]:

test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = keras.preprocessing.image_dataset_from_directory('../input/melspectro-newdata/MelSpectro_1sec_New_Data',
                                                 validation_split=0.99,
                                                 label_mode = "categorical",
                                                 subset="validation", 
                                                 shuffle = False,
                                                 seed=123,
                                                 image_size = (64, 64),
                                                 batch_size = 64)

In [ ]:
results = new_model.evaluate(test_set, batch_size=64)

In [ ]:
predictions = new_model.predict(test_set)

In [ ]:
import numpy as np
y_pred = np.rint(predictions)

In [ ]:
y_pred

In [ ]:
y_true = 0
z = 0
for data, labels in test_set:
    if z == 0:
        y_true = labels.numpy()
        z += 1
    else:
        y_true = np.append(y_true, labels, 0)

In [ ]:
y_true

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1)) * 1.0

In [ ]:
cm

In [ ]:
for i in range(len(cm)):
    su = sum(cm[i])
    print(su)
    for j in range(len(cm)):
        cm[i][j] = cm[i][j] * 1.0 / su

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import sklearn.metrics

In [ ]:
class_labels = list(test_set.class_names)

In [ ]:
df_cm = pd.DataFrame(cm, columns=class_labels, index = class_labels)
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16})# font size

In [ ]:
import sklearn.metrics as metrics

In [ ]:
report = metrics.classification_report(y_true, y_pred, target_names=class_labels)
print(report)